In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Visualização inicial do dataset

Usaremos um conjunto de dados de performance de estudantes em exames

In [2]:
link = 'https://raw.githubusercontent.com/ClarisseAlvarenga/cafezinho_crosstab/main/StudentsPerformance.csv'
dados = pd.read_csv(link)
dados.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [3]:
# shape
dados.shape

(1000, 8)

In [4]:
dados.columns = dados.columns.str.replace(' ', '_')
dados.columns

Index(['gender', 'race/ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course', 'math_score', 'reading_score',
       'writing_score'],
      dtype='object')

## Cruzandos dados com groupby

In [5]:
# usando o groupby
matematica_por_genero = (dados.groupby('gender')['math_score'].agg(['mean', 'median', lambda x: x.mode(), 'std'])).reset_index()
matematica_por_genero.columns = ['genero', 'media', 'mediana', 'moda', 'desvio_padrao']
matematica_por_genero

,genero,media,mediana,moda,desvio_padrao
0,female,63.633205,65,65,15.491453
1,male,68.728216,69,62,14.356277


## Crosstab: fazendo nossa primeira tabela cruzada
[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html)

In [6]:
# cruzando dados de genero por tipo de almoço
# basta simplesmente chamar o pd crosstab e passar os argumentos desejados
# escolha quem será o índice e quem serão as colunas
cross = pd.crosstab(index=dados['gender'], columns=dados['lunch'])
cross

lunch,free/reduced,standard
gender,,
female,189,329
male,166,316


In [7]:
# veja que a função crosstab gera um dataframe
type(cross)

pandas.core.frame.DataFrame

## Contando valores por categoria

In [8]:
cross = pd.crosstab(columns=dados['test_preparation_course'], index=dados['parental_level_of_education'])
cross

test_preparation_course,completed,none
parental_level_of_education,,
associate's degree,82,140
bachelor's degree,46,72
high school,56,140
master's degree,20,39
some college,77,149
some high school,77,102


**Transformando a contagem em porcentagem**

In [9]:
pd.crosstab(columns=dados['test_preparation_course'], 
            index=dados['parental_level_of_education'], normalize=True)*100

test_preparation_course,completed,none
parental_level_of_education,,
associate's degree,8.2,14.0
bachelor's degree,4.6,7.2
high school,5.6,14.0
master's degree,2.0,3.9
some college,7.7,14.9
some high school,7.7,10.2


In [73]:
# veja que podemos passar o nome do eixo como argumento de normalize e 
# ele nos dará a porcentagem de acordo com esse eixo
pd.crosstab(columns=dados['test_preparation_course'], 
            index=dados['parental_level_of_education'], normalize='index')*100

test_preparation_course,completed,none
parental_level_of_education,,
associate's degree,36.936937,63.063063
bachelor's degree,38.983051,61.016949
high school,28.571429,71.428571
master's degree,33.898305,66.101695
some college,34.070796,65.929204
some high school,43.016760,56.983240


 **Cruzando mais de uma coluna**

In [10]:
pd.crosstab(index=dados['test_preparation_course'], columns=[dados['gender'], 
                                                             dados['parental_level_of_education']])

gender                                  female                                \
parental_level_of_education associate's degree bachelor's degree high school   
test_preparation_course                                                        
completed                                   42                22          29   
none                                        74                41          65   

gender                                                                     \
parental_level_of_education master's degree some college some high school   
test_preparation_course                                                     
completed                                14           42               35   
none                                     22           76               56   

gender                                    male                                \
parental_level_of_education associate's degree bachelor's degree high school   
test_preparation_course                                                        
completed                                   40                24          27   
none                                        66                31          75   

gender                                                                     
parental_level_of_education master's degree some college some high school  
test_preparation_course                                                    
completed                                 6           35               42  
none                                     17           73               46

>Veja que basta a gente chamar as colunas individualmente dentro de uma lista do argumento 'columns'

In [11]:
# podemos também colocar atribuir lista ao índice para termos um multiíndice
pd.crosstab(index=[dados['gender'], dados['parental_level_of_education']], 
            columns=dados['test_preparation_course'])

test_preparation_course             completed  none
gender parental_level_of_education                 
female associate's degree                  42    74
       bachelor's degree                   22    41
       high school                         29    65
       master's degree                     14    22
       some college                        42    76
       some high school                    35    56
male   associate's degree                  40    66
       bachelor's degree                   24    31
       high school                         27    75
       master's degree                      6    17
       some college                        35    73
       some high school                    42    46

## Agregando funções

In [12]:
# colunas
dados.columns

Index(['gender', 'race/ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course', 'math_score', 'reading_score',
       'writing_score'],
      dtype='object')

In [14]:
# ao passar mais de um cálculo ao aggfunc, veja que devemos colocá-los dentro de uma tupla
# também é necessário passar o argumento values para que o crosstab saiba onde aplicar as funções do aggfunc
math_gender_lunch = pd.crosstab(index=dados['gender'], columns=dados['lunch'], 
                                aggfunc=('mean', 'median', 'std'), values=dados['math_score'])
math_gender_lunch

mean                  median                   std           
lunch  free/reduced   standard free/reduced standard free/reduced   standard
gender                                                                      
female    55.814815  68.124620         57.0     67.0    15.481375  13.617588
male      62.457831  72.022152         62.0     72.0    14.012809  13.425969

In [19]:
# veja que podemos transpor usando .T caso tenhamos muitas colunas
math_gender_lunch = pd.crosstab(index=dados['gender'], 
                                columns=[dados['lunch'], dados['race/ethnicity']], 
                                aggfunc=('mean', 'median', 'std'), values=dados['math_score'])
math_gender_lunch.T

gender                                 female       male
       lunch        race/ethnicity                      
mean   free/reduced group A         49.928571  58.590909
                    group B         56.512821  58.633333
                    group C         52.838710  60.673077
                    group D         58.039216  64.681818
                    group E         61.304348  73.277778
       standard     group A         64.000000  67.387097
                    group B         64.338462  69.839286
                    group C         66.864407  71.758621
                    group D         69.961538  71.752809
                    group E         75.565217  77.924528
median free/reduced group A         48.000000  59.500000
                    group B         58.000000  58.500000
                    group C         53.500000  61.000000
                    group D         58.000000  63.500000
                    group E         61.000000  77.500000
       standard     group A         60.000000  66.000000
                    group B         65.000000  68.500000
                    group C         66.000000  71.000000
                    group D         69.000000  73.000000
                    group E         74.500000  81.000000
std    free/reduced group A         12.079635  14.568914
                    group B         17.027643  14.329818
                    group C         14.931750  12.377591
                    group D         14.974593  11.815978
                    group E         15.569302  16.997597
       standard     group A         12.769010  13.554034
                    group B         15.160884  12.524560
                    group C         12.648378  13.461371
                    group D         11.464731  13.123782
                    group E         14.565793  13.235804

In [49]:
# aqui vamos fazer outro crosstab, agora com genero e raça
# também usamos o parâmetro margins, que calcula o aggfunc por linha e coluna. Nese caso, soma
# o parâmetro margins_name dá um novo nome para a coluna gerada a partir do parâmetro 'margins'
math_gender_ethnicity = pd.crosstab(columns=dados['gender'], index=dados['race/ethnicity'], 
                                    aggfunc=('sum'), values=dados['math_score'], margins=True, margins_name='Soma')
math_gender_ethnicity

gender,female,male,Soma
race/ethnicity,,,
group A,2107,3378,5485
group B,6386,5670,12056
group C,11166,9398,20564
group D,8417,9232,17649
group E,4886,5449,10335
Soma,32962,33127,66089


Importante notar que quando não usamos o parâmetro aggfunc, a função pd.crosstab faz uma contagem de frequência dos dados.
<br>

Veja abaixo:

In [56]:
pd.crosstab(index=dados['gender'], columns=dados['race/ethnicity'], margins=True)

race/ethnicity,group A,group B,group C,group D,group E,All
gender,,,,,,
female,36,104,180,129,69,518
male,53,86,139,133,71,482
All,89,190,319,262,140,1000


>Aqui ele conta o número de indivíduos por gênero e por raça

In [72]:
cross = pd.crosstab(columns=dados['race/ethnicity'], index=dados['parental_level_of_education'], margins=True)
cross

race/ethnicity,group A,group B,group C,group D,group E,All
parental_level_of_education,,,,,,
associate's degree,14,41,78,50,39,222
bachelor's degree,12,20,40,28,18,118
high school,18,48,64,44,22,196
master's degree,3,6,19,23,8,59
some college,18,37,69,67,35,226
some high school,24,38,49,50,18,179
All,89,190,319,262,140,1000
